# Function Calling

Using the OpenAI function calling interface

## OpenAI Example

In [1]:
import os
import json
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

/Users/altaf/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)


def run_conversation():
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": "What's the weather like in Boston?"}]
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    response_message = response["choices"][0]["message"]

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = fuction_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        return second_response


print(run_conversation())

{
  "id": "chatcmpl-7n3SR6bSfyeInNINJJu36uArHPrzO",
  "object": "chat.completion",
  "created": 1691925719,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The weather in Boston is currently sunny and windy with a temperature of 72 degrees Fahrenheit."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 72,
    "completion_tokens": 18,
    "total_tokens": 90
  }
}


## My Test1 

In [3]:
def get_name_info( surname , firstname  , othername):

    name_info = {
        "surname" : surname,
        "firstname" : firstname,
        "othername" : othername,
        "fullname"  : firstname + othername + surname
    }

    return json.dumps( name_info )

In [4]:
def run_conversation_1(messages):
    # Step 1: send the conversation and available functions to GPT
    
    functions = [
        {
            "name": "get_name_info",
            "description": "Extract the information from a given name",
            "parameters": {
                "type": "object",
                "properties": {
                    "surname": {
                        "type": "string",
                        "description": "The last part of the name",
                    },
                    "firstname": {
                        "type": "string",
                        "description": "The first part of the name",
                    },
                    "othername": {
                        "type": "string",
                        "description": "The middle part of the name i.e. not surname or firstname",
                    }

                },
                "required": ["surname" , "firstname" , "othername"],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )
    return response["choices"][0]["message"]

In [5]:
messages = [{"role": "user", "content": "The customer name is Watson John Smith"}]

response_message = run_conversation_1(messages)

print( response_message )

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_name_info",
    "arguments": "{\n  \"surname\": \"Smith\",\n  \"firstname\": \"John\",\n  \"othername\": \"Watson\"\n}"
  }
}


In [6]:
def run_conversation_2(messages , response_message):
# Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_name_info": get_name_info,
        }  # only one function in this example, but you can have multiple
        function_name = response_message["function_call"]["name"]
        fuction_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = fuction_to_call(
            surname=function_args.get("surname"),
            firstname=function_args.get("firstname"),
            othername=function_args.get("othername")
        )

        # Step 4: send the info on the function call and function response to GPT
        messages.append(response_message)  # extend conversation with assistant's reply
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        return second_response




In [7]:
print(run_conversation_2(messages , response_message))

{
  "id": "chatcmpl-7n3V9XOaSKsgapbaUSSImj9zLKRSU",
  "object": "chat.completion",
  "created": 1691925887,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The customer's full name is John Watson Smith."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 84,
    "completion_tokens": 10,
    "total_tokens": 94
  }
}
